pull the unniversity as well

chatgpt where it is

add 2017-2023

In [4]:
import requests
import pandas as pd

# Step 1: Query Crossref for a broad set of results (you can refine this as needed)
URL = "https://api.crossref.org/works?rows=1000"
response = requests.get(URL)
data = response.json()

# Extract subjects from the returned works and add to a set for uniqueness
subjects_set = set()
for item in data['message']['items']:
    if 'subject' in item:
        subjects_set.update(item['subject'])

# Convert the set to a list
subjects_list = list(subjects_set)
subjects = subjects_list
rows = 1000

# Define the years you want to pull data for
years_to_pull = [2017, 2018, 2019, 2020, 2021, 2022, 2023]  

def get_articles_by_subject(subject, year, rows=rows):
    base_url = 'https://api.crossref.org/works'
    params = {
        'filter': f'from-pub-date:{year}-01-01,until-pub-date:{year}-12-31',
        'sort': 'relevance',
        'rows': rows,
        'query.bibliographic': subject
    }

    articles = []
    cursor = '*'

    while cursor:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            items = data.get('message', {}).get('items', [])

            if not items:
                break

            articles.extend(items)
            cursor = data.get('message', {}).get('next-cursor', '')
        else:
            print(f"Request failed with status code: {response.status_code}")
            print(response.content)
            break

    return articles

# Initialize an empty list to store DataFrames for each subject
subject_dataframes = []

# Loop through each subject and fetch data  
for subject in subjects_list:
    try:
        all_articles = []
        for year in years_to_pull:
            articles = get_articles_by_subject(subject, year, rows=rows)
            all_articles.extend(articles)

            data = {
                'DOI': [],
                'Title': [],
                'Container Title': [],
                'Publisher': [],
                'Publish Date': [],
                'Author First Name': [],
                'Author Last Name': [],
                'Author Order': [],
                'Referenced By': []
            }

            for article in all_articles:
                doi = article.get('DOI', '')
                title = article.get('title', [''])[0]
                container_title = article.get('container-title', [''])[0]
                publisher = article.get('publisher', '')
                publish_date = article.get('published-print', {}).get('date-parts', [[]])[0]
                authors = article.get('author', [])
                referenced_by = article.get('is-referenced-by-count', 0)

                for order, author in enumerate(authors, start=1):
                    first_name = author.get('given', '')
                    last_name = author.get('family', '')

                    data['DOI'].append(doi)
                    data['Title'].append(title)
                    data['Container Title'].append(container_title)
                    data['Publisher'].append(publisher)
                    data['Publish Date'].append(publish_date)
                    data['Author First Name'].append(first_name)
                    data['Author Last Name'].append(last_name)
                    data['Author Order'].append(order)
                    data['Referenced By'].append(referenced_by)

            subject_df = pd.DataFrame(data)
            subject_df.columns = ['DOI', 'Title', 'Container Title', 'Publisher', 'Publish Date', 'Author First Name', 'Author Last Name', 'Author Order', 'Referenced By']
            subject_df['Subject'] = subject  # Add a Subject column with the current subject
            subject_dataframes.append(subject_df)

    except Exception as e:
        print(f"Error processing subject '{subject}': {str(e)}")
        continue

# Concatenate all DataFrames into one big DataFrame
massive_Crossreff_pull = pd.concat(subject_dataframes, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
massive_Crossreff_pull.to_csv('massive_Crossreff_pull.csv', index=False)
massive_Crossreff_pull.describe()

In [15]:
massive_Crossreff_pull.shape

(5997846, 10)

In [5]:
# Count the unique titles
unique_title_count = massive_Crossreff_pull['Title'].nunique()
print(f"Unique Titles: {unique_title_count}")

Unique Titles: 651057


In [6]:
title_author_counts = massive_Crossreff_pull.groupby('Title')['Author Last Name'].nunique().reset_index()

# Rename the columns for clarity
title_author_counts.columns = ['Title', 'Unique Author Count']

# Calculate descriptive statistics
statistics = title_author_counts['Unique Author Count'].describe()

# Print the results
print("Descriptive Statistics for Unique Author Counts per Title:")
print(statistics)

Descriptive Statistics for Unique Author Counts per Title:
count    651057.000000
mean          3.184300
std           3.680977
min           1.000000
25%           2.000000
50%           3.000000
75%           4.000000
max        1566.000000
Name: Unique Author Count, dtype: float64


In [9]:
# Group by subject and count unique titles in each subject
subject_title_counts = massive_Crossreff_pull.groupby('Subject')['Title'].nunique().reset_index()
subject_title_counts.columns = ['Subject', 'Unique Title Count']

# Calculate descriptive statistics for unique title counts per subject
subject_stats = subject_title_counts['Unique Title Count'].describe()

# Print the descriptive statistics for unique title counts per subject
print("Descriptive Statistics for Unique Title Counts per Subject:")
print(subject_stats)

Descriptive Statistics for Unique Title Counts per Subject:
count     234.000000
mean     3088.423077
std       574.991364
min       519.000000
25%      2851.000000
50%      3264.500000
75%      3490.500000
max      3907.000000
Name: Unique Title Count, dtype: float64


In [4]:
# Count the null values in each column
null_counts = massive_Crossreff_pull.isnull().sum()

# Print the null counts for each column
print("Null Value Counts for Each Column:")
print(null_counts)

Null Value Counts for Each Column:
DOI                  0
Title                0
Container Title      0
Publisher            0
Publish Date         0
Author First Name    0
Author Last Name     0
Author Order         0
Referenced By        0
Subject              0
dtype: int64


In [16]:
# Calculate the total count of characters in the 'Author First Name' column for each row
total_character_count = massive_Crossreff_pull['Author First Name'].str.len().sum()

# Calculate the average number of characters in the 'Author First Name' column per row
average_character_count = massive_Crossreff_pull['Author First Name'].str.len().mean()

# Calculate the unique count of characters in the 'Author First Name' column
non_null_count = massive_Crossreff_pull['Author First Name'].count()

# Print the results
print(f"Total Character Count in 'Author First Name' column: {total_character_count}")
print(f"Average Character Count in 'Author First Name' column per row: {average_character_count:.2f}")
print(f"Number of Non-Null Values in 'Author First Name' column: {non_null_count}")

Total Character Count in 'Author First Name' column: 37486849
Average Character Count in 'Author First Name' column per row: 6.25
Number of Non-Null Values in 'Author First Name' column: 5997846


###########################################

In [14]:
list(subjects_set)[10:20]

['Anesthesiology and Pain Medicine',
 'Radiation',
 'Industrial and Manufacturing Engineering',
 'Computational Theory and Mathematics',
 'Library and Information Sciences',
 'Earth-Surface Processes',
 'Biomedical Engineering',
 'Visual Arts and Performing Arts',
 'Biomaterials',
 'Leadership and Management']